# Scenario 2 - ESMValCore

This notebook proposes a design for the ESMValTool/ESMValCore API. The code in this notebook has been heavily mocked to demonstrate the potential usage.

In this scenario, we look at the API from a recipe developers perspective, a climate scientist who is developing his/her own code to perform some calculations and wants to implement those using ESMValTool so that it can be easily shared with others.

For this example, we want to create the climwip diagnostic from scratch, define the datasets and variables, apply preprocessor functions, and save the workflow as a recipe. 

**Steps**
- Reconstruct ClimWIP recipe from scratch using API calls like (see ESMValCore [#489](https://github.com/ESMValGroup/ESMValCore/issues/498)):
- Define datasets/datasetlists
- Apply preprocessor functions
- Call diagnostic script
- Save workflow as recipe

In [1]:
%load_ext autoreload
%autoreload 2

First, let's import the api and initialize an empty recipe.

In [2]:
from esmvaltool_mockup import *

## Datasets

The datasets in ESMValTool are split into two parts. The shared part (usually defined by the `datasets` keyword in the recipe) and the variable group (usually defined as part of the `variables` section in the diagnostics). While this makes sense in the context of keeping the recipes short, we have more flexibility in Python to define this using dictionaries. The combination of the variable / dataset is refered to as a `DatasetReference` here. It refers to a dataset, but is only realized later on.

We want to define two lists of datasets, one for `tas`...

In [3]:
dataset_tas_1 = DatasetReference(
    dataset='ACCESS1-0',
    project='CMIP5',
    exp=['historical', 'rcp45'],
    ensemble='r1i1p1',
    short_name='tas',
    start_year=1995,
    end_year=2014,
    mip='Amon'
)

dataset_tas_2 = DatasetReference(
    dataset='ACCESS1-0',
    project='CMIP5',
    exp=['historical', 'rcp85'],
    ensemble='r1i1p1',
    short_name='tas',
    start_year=1995,
    end_year=2014,
    mip='Amon'
)

The dataset decriptions can be used to locate the data.

In [4]:
dataset_tas_1.locate()

'~/data/CMIP5/tas_ACCESS1-0_CMIP5_Amon_historical-rcp45_r1i1p1_1995-2014.nc'

And, the locations can be used to check if the data are available.

In [5]:
dataset_tas_1.is_available()  # True/False

True

To work with the data, add them to a `DatasetList`. This object knows how to compute and work with the data, and can be used to apply preprocessors to.

In [6]:
datasets_tas = DatasetList(dataset_tas_1, dataset_tas_2)
datasets_tas

DatasetList('NewDatasetList',
queue=(PreprocessorStep('load_data'),
 PreprocessorStep('cmor_check')),
datasets=(DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp45'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995}), DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp85'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995})))

If you want to stop here, you can get the data from the `DatasetList` directly. This will run the required fixes / checks and return the data as iris cubes.

In [7]:
datasets_tas.get_cubes()

Running PreprocessorStep('load_data')
Running PreprocessorStep('cmor_check')


'Cubes'

And for `pr`... This being Python, we can isolate the common parameters into a seperate dictionary, and use dictionary expansion to add the keys to the dataset.

In [8]:
common_keys = {
 'dataset': 'ACCESS1-0',
 'project': 'CMIP5',
 'ensemble': 'r1i1p1',
 'short_name': 'tas',
 'start_year': 1995,
 'end_year': 2014,
 'mip': 'Amon'
}

dataset_pr_1 = DatasetReference(exp=['historical', 'rcp45'], **common_keys)
dataset_pr_2 = DatasetReference(exp=['historical', 'rcp85'], **common_keys)

datasets_pr = DatasetList(dataset_pr_1, dataset_pr_2)
datasets_pr

DatasetList('NewDatasetList',
queue=(PreprocessorStep('load_data'),
 PreprocessorStep('cmor_check')),
datasets=(DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp45'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995}), DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp85'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995})))

## Preprocessors

There are several ways to declare the preprocessors. The most basic way most closely represents the recipe `.yml` format.

In [9]:
steps = {
    'regrid': {
      'target_grid': '2.5x2.5',
      'scheme': 'linear',
    },
    'mask_landsea': {
      'mask_out': 'sea',
    },
    'extract_region': {
      'start_longitude': -10.0,
      'end_longitude': 39.0,
      'start_latitude': 30.0,
      'end_latitude': 76.25,
    }
}

climwip_general = Preprocessor(name='climwip_general', steps=steps)
climwip_general

Preprocessor('climwip_general')
- regrid
- mask_landsea
- extract_region

Although it is possible to use a custom order using `custom_order=True`, ESMValTool tries to determine the scientifically most sensible order from the steps given. This order can be retrieved using the `.order` attribute.

In [10]:
climwip_general.order

['regrid', 'mask_landsea', 'extract_region']

To inspect each individual `PreprocessorStep` and its settings, note that `.steps` is a dictionary.

In [11]:
climwip_general.steps['regrid']

PreprocessorStep('regrid', target_grid=2.5x2.5, scheme=linear)

Now that the steps have been defined once for `climwip_general`, we can use those as a basis for the `climatological_mean` preprocessor, which adds a `climate_statistics` step.

In [12]:
climate_statistics_mean = PreprocessorStep(function='climate_statistics', operator='mean')

steps = climwip_general.steps.copy()  # make a copy
steps['climate_statistics'] = climate_statistics_mean
steps

{'regrid': PreprocessorStep('regrid', target_grid=2.5x2.5, scheme=linear),
 'mask_landsea': PreprocessorStep('mask_landsea', mask_out=sea),
 'extract_region': PreprocessorStep('extract_region', start_longitude=-10.0, end_longitude=39.0, start_latitude=30.0, end_latitude=76.25),
 'climate_statistics': PreprocessorStep('climate_statistics', operator=mean)}

Once the steps have been defined, a new `Preprocessor` can be constructed.

In [13]:
climatological_mean = Preprocessor(name='climatological_mean', steps=steps)

Even more straightforward is to consider `climwip_general` as a Preprocessor step by itself, so a shortcut for the code above would be to pass it as a step by itself.

In [14]:
detrend = PreprocessorStep(
    function='detrend', 
    dimension='time', 
    method='linear'
)

climate_statistics_std = PreprocessorStep(
    function='climate_statistics', 
    operator='std_dev'
)

detrended_std = Preprocessor(
    name='detrended_std', 
    steps=[climwip_general, detrend, climate_statistics_std]
)

Finally, we may be lazy and already have some yaml code lying around from another recipe that we want to use. For this, we can use the `.from_yaml` classmethod.

In [15]:
yaml_code = """
  temperature_anomalies:
    area_statistics:
      operator: mean
    annual_statistics:
      operator: mean
    anomalies:
      period: full
      reference:
        start_year: 1981
        start_month: 1
        start_day: 1
        end_year: 2010
        end_month: 12
        end_day: 31
      standardize: false
"""
      
preproc = Preprocessor.from_yaml(yaml_code)
preproc

Preprocessor('temperature_anomalies')
- area_statistics
- annual_statistics
- anomalies

## Applying a preprocessor

`DatasetLists` are used to play with variables and preprocessor. Preprocessors/steps can be applied directly to a `DatasetList`. This can happen in place, or when a new instance of the object is returned (with `in_place=False`). This allows datasets to be forked re-used. The steps are tracked in a *steps queue*.

Here, the `DatasetList` is inialized with the the `climwip_general` preprocessor. The steps are applied when `.compute` is called. Note that the `queue` is then empty.

In [16]:
ds_pr = DatasetList(
    datasets_pr,
    name='my first dataset',
    preprocessor=climwip_general,
)

ds_pr.compute()
ds_pr

Running PreprocessorStep('load_data')
Running PreprocessorStep('cmor_check')
Running Preprocessor('climwip_general')
 -> PreprocessorStep('regrid')
 -> PreprocessorStep('mask_landsea')
 -> PreprocessorStep('extract_region')


DatasetList('my first dataset',
queue=(),
datasets=(DatasetList('NewDatasetList',
queue=(PreprocessorStep('load_data'),
 PreprocessorStep('cmor_check')),
datasets=(DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp45'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995}), DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp85'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995}))),))

Alternatively, the data can be initialized using the individual datasets and without a preprocessor. In this example, the steps are added one-by-one and new `DatasetList` objects are returned. This gives some more flexibility to fork the items. Note that the queue is still full.

In [17]:
ds_pr = DatasetList(dataset_pr_1, dataset_pr_2)

ds_regridded = ds_pr.add_step(
    function='regrid',
    target_grid='2.5x2.5', 
    scheme='linear',
    in_place=False,
)  # lazy

ds_masked = ds_regridded.add_step(
    function='mask_landsea',
    mask_out='sea',
    in_place=False,
)

ds_region = ds_masked.add_step(
    function='extract_region',
    start_longitude= -10.0, 
    end_longitude= 39.0,
    start_latitude= 30.0,
    end_latitude= 76.25,
    in_place=False,
)

ds_region

DatasetList('NewDatasetList',
queue=(PreprocessorStep('load_data'),
 PreprocessorStep('cmor_check'),
 PreprocessorStep('regrid'),
 PreprocessorStep('mask_landsea'),
 PreprocessorStep('extract_region')),
datasets=(DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp45'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995}), DatasetReference({'dataset': 'ACCESS1-0',
 'end_year': 2014,
 'ensemble': 'r1i1p1',
 'exp': ['historical', 'rcp85'],
 'mip': 'Amon',
 'project': 'CMIP5',
 'short_name': 'tas',
 'start_year': 1995})))

The queue can be printed using `.print_q`, and shows the steps that have been performed / are to be done.

In [18]:
ds_region.print_queue()

Steps queue
-----------
Todo -> PreprocessorStep('load_data')
Todo -> PreprocessorStep('cmor_check')
Todo -> PreprocessorStep('regrid')
Todo -> PreprocessorStep('mask_landsea')
Todo -> PreprocessorStep('extract_region')



The operation is lazy, and execution is delayed until the data are requested.

In [19]:
ds_region.compute()            # consume steps, optional
cubes = ds_region.get_cubes()  # return data cubes
cubes

Running PreprocessorStep('load_data')
Running PreprocessorStep('cmor_check')
Running PreprocessorStep('regrid')
Running PreprocessorStep('mask_landsea')
Running PreprocessorStep('extract_region')


'Cubes'

The performed steps are now marked as *Done*.

In [20]:
ds_region.print_queue()

Steps queue
-----------
Done -> PreprocessorStep('load_data')
Done -> PreprocessorStep('cmor_check')
Done -> PreprocessorStep('regrid')
Done -> PreprocessorStep('mask_landsea')
Done -> PreprocessorStep('extract_region')



This means that the cubes are cached on this item. Asking for the cubes again, will return the same object.

In [21]:
ds_region.get_cubes()

'Cubes'

This also means that new steps can be easily added, as the intermediate result is stored. For example, to recreate the `climatological_mean` preprocessor, we apply the `climate_statistics_mean` step defined before.

In [22]:
climate_statistics_mean

PreprocessorStep('climate_statistics', operator=mean)

In [23]:
climstats = ds_region.add_step(climate_statistics_mean, in_place=False)
climstats.print_queue()

Steps queue
-----------
Done -> PreprocessorStep('load_data')
Done -> PreprocessorStep('cmor_check')
Done -> PreprocessorStep('regrid')
Done -> PreprocessorStep('mask_landsea')
Done -> PreprocessorStep('extract_region')
Todo -> PreprocessorStep('climate_statistics')



Note how three steps are already marked as `Done`. Requesting the data only executes the last step.

In [24]:
cubes = climstats.get_cubes()

Running PreprocessorStep('climate_statistics')


Most imporantly, the state of our original dataset list has not been changed, so we can still use that for some other data processing.

In [25]:
ds_pr.print_queue()

Steps queue
-----------
Todo -> PreprocessorStep('load_data')
Todo -> PreprocessorStep('cmor_check')



It is also possible to apply a preprocessor directly.

In [26]:
data_detrended = ds_pr.add_step(detrended_std, in_place=False)

data_detrended.print_queue()

cubes = data_detrended.get_cubes()
cubes

Steps queue
-----------
Todo -> PreprocessorStep('load_data')
Todo -> PreprocessorStep('cmor_check')
Todo -> Preprocessor('detrended_std')

Running PreprocessorStep('load_data')
Running PreprocessorStep('cmor_check')
Running Preprocessor('detrended_std')
 -> PreprocessorStep('regrid')
 -> PreprocessorStep('mask_landsea')
 -> PreprocessorStep('extract_region')
 -> PreprocessorStep('detrend')
 -> PreprocessorStep('climate_statistics')


'Cubes'

## Diagnostic

The diagnostic consists of the name of a script to call, as well as the ancestors to use. Although in Python we can call `.get_cubes` and play with the data directly, for the purpose of this notebook we want to use the data to call a diagnostic.
The diagnostic must know on which data to run. These are defined by the ancestors.

In [27]:
ds_tas = DatasetList(
    dataset_tas_1, dataset_tas_2,
    name='dataset_tas',
    preprocessor=detrended_std,
)

ds_pr = DatasetList(
    dataset_pr_1, dataset_pr_2,
    name='dataset_pr',
    preprocessor=climatological_mean,
)

ancestors = ds_tas, ds_pr

We specify the name of the script we want to run `weighting/climwip.py`. The remaining parameters are passed to the script directly.

In [28]:
shape_params = {
  'tas': {
    'sigma_d': 0.588,
    'sigma_s': 0.704,
  },
  'pr': {
    'sigma_d': 0.658,
    'sigma_s': 0.704, 
  }
}

calculate_weights_climwip = Diagnostic(
    name='climwip',
    script='weighting/climwip.py', 
    ancestors=ancestors, 
    shape_params=shape_params,
)

We can run a diagnostic directly, and pass the ancestors directly to the script. ESMValTool will run the diagnostic in a new session directory, but if we want to change the session, we can pass the session name. ESMValTool will make a new timestamped directory and store the data there.

In [29]:
calculate_weights_climwip.run(session='diag_climwip')

Session directory: diag_climwip-20201106_153837

Working on 'dataset_tas'
Running PreprocessorStep('load_data')
Running PreprocessorStep('cmor_check')
Running Preprocessor('detrended_std')
 -> PreprocessorStep('regrid')
 -> PreprocessorStep('mask_landsea')
 -> PreprocessorStep('extract_region')
 -> PreprocessorStep('detrend')
 -> PreprocessorStep('climate_statistics')

Working on 'dataset_pr'
Running PreprocessorStep('load_data')
Running PreprocessorStep('cmor_check')
Running Preprocessor('climatological_mean')
 -> PreprocessorStep('regrid')
 -> PreprocessorStep('mask_landsea')
 -> PreprocessorStep('extract_region')
 -> PreprocessorStep('climate_statistics')

Running script 'weighting/climwip.py'


## Building the recipe

In general, a ESMValTool recipe consists of 4 parts:
- documentation
- datasets
- preprocessors
- diagnostics

All but the documentation has already been defined. The key parts here is the diagnostic, which already knows about the preprocessors and the datasets. Although the datasets and preprocessors have a lot of presence in the recipe, they mainly serve to make the diagnostic definition more consise.

Within the Python API, there is much more flexibility to define these, and the preprocessors / datasets are directly attached to the diagnostic.

To put together a recipe, we must initialize a new recipe.

In [30]:
recipe = Recipe('climwip')
recipe

Recipe('climwip')

We can then add the diagnostic(s) to the recipe.

In [31]:
recipe.add_diagnostic('calculate_weights_climwip', calculate_weights_climwip)

## Documentation

Before saving the recipe, we should add the authors using the specified methods.

In [32]:
recipe.set_description("EUCP ClimWIP")
recipe.set_authors(
    'kalverla_peter',
    'smeets_stef',
    'brunner_lukas',
    'camphuijsen_jaro',
)
recipe.set_maintainer(
    'kalverla_peter',
    'smeets_stef',
    'brunner_lukas',
)
recipe.set_references(
    'brunner2019',
    'lorenz2018',
    'knutti2017',
)

Finally, we can save the recipe...

In [33]:
recipe.save('climwip_from_notebook.yml')

Recipe saved to 'climwip_from_notebook.yml'


...and run it!

In [34]:
recipe.run(session='jupyter_recipe')

Session directory: jupyter_recipe-20201106_153845



Starting Diagnostic('weighting/climwip.py')

Working on 'dataset_tas'

Working on 'dataset_pr'

Running script 'weighting/climwip.py'



In [ ]:
recipe.show() # -> should print the recipe, but it does not work yet ;-)